##### encoding=utf-8
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
import gensim
import pickle
from gensim import models
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from gensim.models.ldamodel import LdaModel
from gensim.models.tfidfmodel import TfidfModel

In [4]:
content = open('jd_reviews.txt','rb').read()

In [5]:
tags = jieba.analyse.extract_tags(content, topK=20, withWeight=True)
word_rank = pd.DataFrame(tags,columns=['word','rank'])
word_rank

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Yanbo\AppData\Local\Temp\jieba.cache
Loading model cost 1.108 seconds.
Prefix dict has been built succesfully.


,word,rank
0,不错,0.358839
1,鼠标,0.197843
2,手感,0.162806
3,罗技,0.130808
4,键盘,0.125591
5,好用,0.093429
6,hellip,0.089139
7,京东,0.079309
8,好好,0.072409
9,非常,0.067122


In [6]:
sentences = []
with open('jd_reviews.txt', 'rb') as f:
    sentences += [list(jieba.cut(line.strip())) for line in f]

In [7]:
print (sentences [20:100])

[['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1'], ['1']]


In [8]:
model = gensim.models.Word2Vec(sentences, 
                               size=100, 
                               window=5, 
                               min_count=5, 
                               workers=4)

In [9]:
for k, s in model.most_similar(positive=["鼠标"]):
    print (k, s)

手柄 0.6392351388931274
接收器 0.4924774169921875
鼠 0.47291460633277893
之前 0.4715437889099121
有线 0.4701094627380371
键鼠 0.4606958031654358
信号 0.4604909420013428
小键盘 0.4448607861995697
感觉 0.44323351979255676
原来 0.4418410062789917


In [10]:
def find_relationship(a, b, c):
    d, _ = model.most_similar(positive=[c, b], negative=[a])[0]
    str = ("给定“{}”与“{}”，“{}”和“{}”有类似的关系").format(a, b, c, d)
    print (str)

find_relationship("女生", "鼠标", "男生")

给定“女生”与“鼠标”，“男生”和“微动”有类似的关系


In [11]:
def save(data, file):
    
    fo = open(file, 'wb')
    pickle.dump(data, fo, protocol=2)
    fo.close()

In [12]:
dataset_name = 'jd_logi'
dictionary = Dictionary(sentences)
save(dictionary, '%s_dict.pkl' % dataset_name)
corpus = [dictionary.doc2bow(text) for text in sentences]

In [13]:
# id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt')

In [14]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, update_every=1, chunksize=10000, passes=1)
lda.print_topics(5)

[(0,
  '0.098*"。" + 0.077*"，" + 0.062*"好" + 0.055*"很" + 0.046*"的" + 0.029*"鼠标" + 0.023*"键盘" + 0.019*"用" + 0.015*"了" + 0.013*"手感"'),
 (1,
  '0.142*"，" + 0.046*"不错" + 0.044*"鼠标" + 0.043*"的" + 0.040*"还" + 0.034*"很" + 0.032*"用" + 0.031*"。" + 0.020*"好" + 0.019*"可以"'),
 (2,
  '0.178*" " + 0.147*"！" + 0.057*"很" + 0.042*"好" + 0.035*"不错" + 0.026*";" + 0.025*"&" + 0.024*"的" + 0.023*"非常" + 0.021*"hellip"'),
 (3,
  '0.146*"，" + 0.026*"快" + 0.023*"速度" + 0.020*"?" + 0.020*"不错" + 0.019*"鼠标" + 0.018*"~" + 0.017*"送货" + 0.017*"值得" + 0.016*"的"'),
 (4,
  '0.094*"，" + 0.049*"的" + 0.041*"了" + 0.024*"鼠标" + 0.019*"是" + 0.016*"我" + 0.015*"就" + 0.015*"不" + 0.011*"买" + 0.010*"用"')]